<a href="https://colab.research.google.com/github/nahuelalmeira/poblacionPenal/blob/master/AnalisisYCuracion/DiploDatos_PoblacionPenitenciariaArgentina_AyC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DiploDatos 2019 -  Población Penitenciaria en Argentina




> http://diplodatos.famaf.unc.edu.ar/poblacion-penitenciaria-en-argentina-2002-a-2017/




## Práctico Análisis y Curación de Datos

En este notebook trabajaremos sobre el dataset original extraído del portal de datos abiertos de la nación:

https://datos.gob.ar/dataset/justicia-sistema-nacional-estadisticas-sobre-ejecucion-pena---sneep

El objetivo de este trabajo será el de procesar el conjunto de datos para poder utilizarlos en el análisis exploratorio de los mismos. Los entregables del trabajo serán:

1- Dataset con características similares al entregado para el primer práctico:

https://github.com/camporeale/Datos/raw/master/sneep_2017_diplodatos.zip

2- Notebook con el análisis del dataset y los pasos ejecutados para la limpieza del mismo.

3- Script de python que permita automatizar el procesamiento. Debe poder pasarsele por parámetros la versión 2002-2017 del dataset y ejecutar correctamente.

### Inicialización y carga de datos

In [1]:
# Importamos las librerías necesarias
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sys.path.append('../config')
from mpl_settings_v3 import *

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_columns', 100)

In [2]:
# Cargamos el CSV en un dataframe
input_file = '../datasets/sneep-2017.csv'
if os.path.isfile(input_file):
    url = input_file
    data_raw = pd.read_csv(url, index_col=0)
else:
    url = 'http://datos.jus.gob.ar/dataset/6c03af36-6a1d-4306-b2a8-dd39ad73afb3/resource/af0a64da-6d06-45cf-a86c-de00f09221d8/download/sneep-2017.csv'
    data_raw = pd.read_csv(url, index_col=0)

/home/lucia/.anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Análisis

In [3]:
# Obtenemos la cantidad de filas y columnas
print("Cantidad de registros: ", data_raw.shape[0])
print("Cantidad de columnas: ", data_raw.shape[1])

Cantidad de registros:  85283
Cantidad de columnas:  86


In [4]:
# Listamos las columnas y sus tipos de datos
data_raw.dtypes

provincia_id                                            int64
provincia_descripcion                                  object
establecimiento_id                                      int64
establecimiento_descripcion                            object
edad                                                  float64
genero_id                                               int64
genero_descripcion                                     object
nacionalidad_id                                         int64
nacionalidad_descripcion                               object
estado_civil_id                                       float64
estado_civil_descripcion                               object
nivel_instruccion_id                                  float64
nivel_instruccion_descripcion                          object
ultima_situacion_laboral_id                           float64
ultima_situacion_laboral_descripcion                   object
capacitacion_laboral_al_ingresar_id                   float64
capacita

In [5]:
# Analizamos 2 muestras de filas, usamos transpose() para visualizar los datos con mayor comodidad
data_raw.sample(2).transpose()

anio_censo,2017,2017
provincia_id,1,1
provincia_descripcion,Buenos Aires,Buenos Aires
establecimiento_id,18,216
establecimiento_descripcion,UNIDAD 13 - JUNIN,UNIDAD 38 - SIERRA CHICA
edad,36,30
genero_id,1,1
genero_descripcion,Masculino,Masculino
nacionalidad_id,5,1
nacionalidad_descripcion,Paraguaya,Argentina
estado_civil_id,1,1


### Lineamientos básicos del preprocesamiento 



* Parseo de fechas. Los campos "fecha_detencion" y "fecha_condena" deben ser transformados a tipos de datos date, en este proceso puede que se encuentren errores, analizar los mismos y decidir si conservar o descartar registros con problemas.

* Análisis y Selección de Features. El dataset tiene campos duplicados que expresan la misma información: campos con sufijo "_id" y "_descripcion". En general estos campos son de tipo numérico y string respectivamente, pero hay algunas excepciones. Identificarlas y editar el dataframe para que quede consistente, conservar solo los campos con las descripciones.
 
* Análisis de consistencia de los datos 1: Analizar los valores del campo "mujer_tiene_hijos_intramuro", identificar el problema que afecta a los datos, y corregir el dataset acorde a esto. 

* Análisis de consistencia de los datos 2: Analizar los campos "fecha_detencion" y "fecha_condena"

* Generación de nuevas Features. Generar un nuevo campo que contenga información acerca del tiempo en días que toma la resolución de la situación procesal de cada persona (diferencia entre fecha de encarcelamiento y de condena)

* Análisis e imputación de valores nulos. Existen muchos registros con valores nulos en múltiples columnas. Analizar las posibles soluciones. Tener en cuenta que dependiendo el tipo de trabajo que querramos hacer con los datos en un futuro, es probable que necesitemos realizar tareas como: eliminar registros, eliminar columnas, cargar datos con valores aleatorios, cargar datos con valores más frecuentes, etc. 

* Bonus:  El dataset contiene información respecto a individuos particulares, y si bien no hay nombres o identificadores únicos como el número de documento, es muy posible que se pueda inferir la identidad de muchas personas usando otros conjuntos de datos públicos. Teniendo en cuenta que uno de los campos describe si la persona recibió atención médica por distintos tipos de enfermedades (HIV, Chagas, Hepatitis, etc.), es importante pensar en qué tipo de acciones podrían tomarse en la elaboración del reporte para asegurar la anonimidad de los datos, o de no ser posible, la eliminación de aquellos atributos que podrían ser considerados información sensible y/o privada. Hacer un análisis corto respecto a este tema y exponer sus opiniones.

Primero exploramos un poco el data set para deducir cuales son los datos problemáticos que necesitamos limpiar o curar.

Veamos qué ocurre con la columna 'mujer_tiene_hijos_intramuro'

In [6]:
data_raw['mujer_tiene_hijos_intramuro'].describe()

count    85283.000000
mean        25.078691
std         27.009273
min          2.000000
25%          6.000000
50%          6.000000
75%         50.000000
max         94.000000
Name: mujer_tiene_hijos_intramuro, dtype: float64

In [7]:
data_raw['mujer_tiene_hijos_intramuro'].unique()

array([ 6, 10, 14, 18, 22, 26, 30, 34, 38, 42, 46, 50, 54, 58, 62, 66, 70,
       74, 78, 82, 86, 94, 90,  2])

Vemos que aparecen valores que no tienen sentido, ya que una mujer no puede tener, por ejemplo, 70 o 90 hijos intramuro. Como los datos no son consistentes, deducimos que lo que debe estar ocurriendo es que algunas columnas estén desfasadas y que esos valores correspondan a otras variables. 

Analizamos los campos 'fecha_detencion' y 'fecha_condena'

In [8]:
data_raw['fecha_detencion'].describe()

count                   78643
unique                   5611
top       27/09/2017 00:00:00
freq                      203
Name: fecha_detencion, dtype: object

In [ ]:
data_raw['fecha_detencion'].unique()

array(['23/02/2017 00:00:00', '14/07/2016 00:00:00',
       '06/11/2015 00:00:00', ..., '24/11/2007 00:00:00',
       '15/12/2008 00:00:00', '21/12/1987 00:00:00'], dtype=object)

In [ ]:
data_raw['fecha_detencion'].value_counts().head()

27/09/2017 00:00:00    203
22/12/2017 00:00:00    119
15/06/2017 00:00:00    110
15/12/2016 00:00:00    109
18/08/2017 00:00:00    108
Name: fecha_detencion, dtype: int64

In [ ]:
data_raw['fecha_detencion'].isna().sum()

6640

In [ ]:
data_raw['fecha_condenado'].describe()

count                   38314
unique                   6815
top       20/09/2017 00:00:00
freq                       61
Name: fecha_condenado, dtype: object

In [ ]:
data_raw['fecha_condenado'].unique()

array(['08/11/2017 00:00:00', nan, '12/03/2017 00:00:00', ...,
       '22/08/2008 00:00:00', '01/10/2004 00:00:00',
       '12/11/2003 00:00:00'], dtype=object)

In [ ]:
data_raw['fecha_condenado'].value_counts().tail()

27/12/2025 00:00:00    1
26/04/2027 00:00:00    1
14/03/2024 00:00:00    1
13/11/2026 00:00:00    1
07/03/2015 00:00:00    1
Name: fecha_condenado, dtype: int64

In [ ]:
data_raw['fecha_condenado'].isna().sum()

46969

En principio no parece haber problemas con estas columnas, salvo que hay muchos datos faltantes (NaN). Una posible inconsistencia es que las fechas de detención o condena sean posteriores al año del censo, ya que esto no tendría sentido, vamos a hacer un filtro para este tipo de casos. 

Veamos de qué tipo de datos son estas fechas:

In [ ]:
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

print(is_string_dtype(data_raw['fecha_detencion']))
print(is_numeric_dtype(data_raw['fecha_detencion']))
print(is_string_dtype(data_raw['fecha_condenado']))
print(is_numeric_dtype(data_raw['fecha_condenado']))

True
False
True
False


Vemos que el tipo de datos es string, vamos a convertirlos a un tipo de datos que dispone pandas llamado **datetime**. 

(Fuente: https://www.datacamp.com/community/tutorials/converting-strings-datetime-objects) 

También detectamos otros errores, como una columna '_id' a la que le faltaba este sufijo, arreglaremos esto durante el preprocesamiento, ya que después necesitamos eliminar estas columnas y para eso deben estar bien nombradas.

En el data set entregado la separación se hace por comas, quisieramos cambiar esto a punto y coma, ya que muchas veces hay strings que tienen comas en su interior, pero corresponden a un único campo, por ejemplo el establecimiento "U.R.II - (ALCAIDIA, COMISARIA 2 Y 3º,TRENEL, QUEMU, ALVEAR, REALICO, LUIGGI, CALEUFU-PARERA)" de La Pampa.

A continuación definimos las funciones que necesitamos para limpiar el dataset.

In [ ]:
def preprocesar_csv(input_file, output_file, year=None):
    """
    Cambia separación por coma a separación por punto y coma y elimina comillas dobles. 
    También corrige algunos errores en ciertas columnas.
    """
    
    number_of_fields = []
    with open(input_file, 'r') as in_f:
        with open(output_file, 'w') as out_f:
            line = in_f.readline()
            line = line.replace('recibio_atencion_medica_ult_anio,', 
                                    'recibio_atencion_medica_ult_anio_id,') #col _id mal nombrada 
            if year == 2017:
                line = line.replace('tiene_medidas_seguridad,', '') #col sin datos     
            while line:
                out_line = line.replace(',TRENEL', ', TRENEL')         
                out_line = out_line.replace(', ', '[COMA] ')
                out_line = out_line.replace(',', ';')
                out_line = out_line.replace('[COMA]', ',')
                out_line = out_line.replace(';"', ";")
                out_line = out_line.replace('";', ";")
                NF = len(out_line.split(';'))
                number_of_fields.append(NF)
                out_f.write(out_line)
                line = in_f.readline()
                
    assert len(np.unique(number_of_fields)) == 1

In [ ]:
from datetime import datetime

def filtrar_nulos(df, f=0.8):
    """
    Descarta registros que tengan al menos
    una fraccion f de campos nulos.
    """
    
    n_cols = df.shape[1]
    n_null = df.isnull().sum(axis=1)
    mask = n_null < int(n_cols*f)
    df = df[mask]
    return df
    

def procesar_fechas(df, current_year):
    """
    Descarta fechas que sean posteriores al 31 de Diciembre
    del año en que se realizo el censo.
    """
    
    date_cols = ['fecha_detencion', 'fecha_condenado']
    incorrect_dates = {}
    for date_col in date_cols:
        incorrect_dates[date_col] = 0
        corrected_values = []
        for i, date in df[date_col].items():
            if isinstance(date, str):
                year = int(date.split('/')[2][:4])
                if year > current_year:
                    incorrect_dates[date_col] += 1
                    date = np.NaN
                else:
                    try:
                        date = date[:10]
                    except:
                        print(date)

            corrected_values.append(date)
        df[date_col] = corrected_values

    df['fecha_detencion'] = pd.to_datetime(df['fecha_detencion'])
    df['fecha_condenado'] = pd.to_datetime(df['fecha_condenado'])
    return df

def tipo_fechas(df):
    """
    Cambia el tipo de las fechas de string a datetime
    """
    
    cols = ['fecha_detencion', 'fecha_condenado']
        
    for col in cols:
        df[col] =  pd.to_datetime(df[col], format='%m/%d/%y')
    return df

def calcular_tiempo_resolucion(df):
    """
    Crea un nuevo campo con el tiempo de resolucion de condena.
    """
    
    condenados = df['situacion_legal_descripcion'] == 'Condenado'
    df['tiempo_resolucion'] = df['fecha_condenado'] - df['fecha_detencion']
    return df

def calcular_numero_delitos(df):
    """
    Crea un nuevo campo con la cantidad de delitos cometidos.
    """
    
    delito_cols = ['delito{}_descripcion'.format(i) for i in range(1, 6)]
    df['delitos_cantidad'] = df[delito_cols].count(axis=1)
    return df

def procesar_edades(df):
    """
    Reemplaza los NaN y valores iguales a 0 por la media de las edades.
    """
    
    #mask = (df['edad'] == 0) & ((df['delitos_cantidad'] > 0) | \
    #                              (df['situacion_legal_descripcion'] == 'Condenado') | \
    #                              (df['nivel_instruccion_descripcion'].notnull()))
    mask = df['edad'] == 0
    df[mask] = np.NaN
    df['edad'].fillna(df['edad'].mean(skipna=True))
    return df

def eliminar_id(df):
    """
    Elimina las columnas _id, nos quedamos solo con las descripciones
    """
    df = df.drop([x for x in df if x.endswith('_id')], 1)
    df.columns = [col.replace('_descripcion', '') for col in df.columns]
    return df

In [ ]:
input_file = '../datasets/sneep-2017.csv'
output_file = '../datasets/py_sneep-2017.csv'
year = 2017

print('Preprocesando csv')
preprocesar_csv(input_file, output_file, year)

df = pd.read_csv(output_file, sep=';')

print('Procesando fechas')
df = procesar_fechas(df, current_year=year)
df = tipo_fechas(df)
print('Calculando tiempo resolucion')
df = calcular_tiempo_resolucion(df)
print('Calculando numero de delitos')
df = calcular_numero_delitos(df)
print('Procesando edades')
df = procesar_edades(df)
print('Eliminando filas con pocos datos')
df = filtrar_nulos(df, f=0.7)
print('Eliminando las columnas _id' )
df = eliminar_id(df)

df.to_csv(path_or_buf=output_file, sep=';')

Preprocesando csv
Procesando fechas
Calculando tiempo resolucion
Calculando numero de delitos
Procesando edades
Eliminando filas con pocos datos
Eliminando las columnas _id


In [ ]:
df.shape

(84914, 51)

Tenemos menos columnas ya que eliminamos las '_id' y otras. También tenemos menos filas (las que tenían más del 70% de los campos sin información).

In [ ]:
# Listamos las columnas y sus tipos de datos
df.dtypes

Veamos cómo quedó la columna 'mujer_tiene_hijos_intramuro'

In [ ]:
df['mujer_tiene_hijos_intramuro'].describe()

In [ ]:
df['mujer_tiene_hijos_intramuro'].unique()

In [ ]:
df['mujer_tiene_hijos_intramuro'].value_counts()

In [ ]:
df['mujer_tiene_hijos_intramuro'].isna().sum()

Se solucionó el problema que teníamos y ahora los datos son consistentes. 

In [ ]:
no_nulls = df['mujer_tiene_hijos_intramuro'].value_counts().sum()
nulls = df['mujer_tiene_hijos_intramuro'].isna().sum()
prop_nulls = nulls/(nulls + no_nulls)
print(prop_nulls)

Se observa que hay casi un 96% de valores Nan, esto tiene sentido, ya que este porcentaje corresponde a la población masculina, que no tiene datos en este campo.

Veamos cómo quedaron las fechas:

In [ ]:
print(is_string_dtype(df['fecha_detencion']))
print(is_numeric_dtype(df['fecha_detencion']))
print(is_string_dtype(df['fecha_condenado']))
print(is_numeric_dtype(df['fecha_condenado']))

In [ ]:
df.select_dtypes(include=[np.datetime64]).head()

In [ ]:
df['fecha_condenado'].tail()

Como podemos ver, ahora son todas del tipo **datetime**.

### Anonimización de los datos

Una forma de identificar a un sujeto de este data set puede ser justamente por la fecha de detención o la fecha de condena. Con el siguiente código vemos que solo en el año 2017, hay 1290 fechas de detención únicas (osea que se repiten una sola vez) y 1180 fechas de condena únicas. Estas personas podrían ser facilmente identificadas. 

In [ ]:
vc = df['fecha_detencion'].value_counts()
vc[vc <= 1]

In [ ]:
vc = df['fecha_condenado'].value_counts()
vc[vc <= 1]

Una forma de evitar esto es, dado que a nosotros no nos interesa la fecha exacta de detención y condena, en cambio nos interesa el tiempo transcurrido entre una y otra, y ya hemos creado una columna con esta información ('tiempo_resolucion'), podemos simplemente eliminar estas dos columnas. Otra forma podría ser eliminar el día y quedarnos con solo mes y año, de esta forma los datos serían menos individualizables.